# Segment raw images to get gray matter before normalising.
This is to prevent errors in normalisation down the line (e.g. enlargement of ventricles: GM WM growth or shrinkage?)

In [16]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.spm as spm
import numpy as np
import os

In [2]:
# define useful functions

# get subject directories from parent directory
def get_subdirs(dir):
    subdirs=list()
    for i in os.listdir(dir):
        if os.path.isdir(os.path.join(dir, i)):
            subdirs.append(os.path.join(dir, i))
    return sorted(subdirs)

In [3]:
subdirs=get_subdirs("/VBM/")
print(subdirs)

['/VBM/FED006', '/VBM/FED007', '/VBM/FED008', '/VBM/FED009', '/VBM/FED010', '/VBM/FED011', '/VBM/FED012', '/VBM/FED013', '/VBM/FED014', '/VBM/FED015', '/VBM/FED016', '/VBM/FED017', '/VBM/FED018', '/VBM/FED019', '/VBM/FED020', '/VBM/FED021', '/VBM/FED022', '/VBM/FED023', '/VBM/FED024', '/VBM/FED025', '/VBM/FED026', '/VBM/FED027', '/VBM/FED028', '/VBM/FED029', '/VBM/FED030', '/VBM/FED031', '/VBM/FED032', '/VBM/FED033', '/VBM/FED034', '/VBM/FED035', '/VBM/FED036', '/VBM/FED037', '/VBM/FED038', '/VBM/FED039', '/VBM/FED040', '/VBM/FED041', '/VBM/FED042', '/VBM/FED043', '/VBM/FED044', '/VBM/FED045', '/VBM/FED046', '/VBM/FED047', '/VBM/FED048', '/VBM/FED050', '/VBM/FED051', '/VBM/FED052', '/VBM/FED053', '/VBM/FED054', '/VBM/FED055', '/VBM/FED056', '/VBM/FED057', '/VBM/FED058', '/VBM/FED059', '/VBM/FED060', '/VBM/FED061', '/VBM/FED062', '/VBM/FED063', '/VBM/FED064', '/VBM/FED065', '/VBM/FED066', '/VBM/FED067', '/VBM/FED068']


In [4]:
# make dict with all images as path references
images=dict()
for i in subdirs:
    for j in os.listdir(i):
        if j.endswith(".nii"):
            images[i.rsplit('/', 1)[1]]=os.path.join(i, j)

print(images)

# load these in different DATA FORMATS (nibabel, pandas) later on

{'FED006': '/VBM/FED006/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii', 'FED007': '/VBM/FED007/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141107104226_5.nii', 'FED008': '/VBM/FED008/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141112155009_5.nii', 'FED009': '/VBM/FED009/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141117154954_5.nii', 'FED010': '/VBM/FED010/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141119091904_5.nii', 'FED011': '/VBM/FED011/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141119161708_5.nii', 'FED012': '/VBM/FED012/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141124150148_5.nii', 'FED013': '/VBM/FED013/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141125093955_5.nii', 'FED014': '/VBM/FED014/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141125153216_5.nii', 'FED015': '/VBM/FED015/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141202145526_5.nii', 'FED016': '/VBM/FED016/T1_MPRAGE_SAG_ISO_0_9_0005

In [23]:
# crop down images for testing (only FED006!!!)
images2=dict()
for i in sorted(images.keys())[0:1]:
    images2[i]=images[i]

In [14]:
# create SKULLSTRIP segment node
# need to perform segmentation with full tissue information, or brain will be classified as "soft tissue" etc. -> see examples in "/VBM/FED006/"
segm_skullstrip_T1=Node(spm.preprocess.NewSegment(), name='segm_skullstrip_T1')
tissue1 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 1), 2, (False,False), (False, False)) # GM
tissue2 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 2), 2, (False,False), (False, False)) # WM
tissue3 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 3), 2, (False,False), (False, False)) # CSF
tissue4 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 4), 2, (True,True), (False, False)) # bone
tissue5 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 5), 2, (True,True), (False, False)) # soft tissue
tissue6 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 6), 2, (False,False), (False, False)) # air and background
segm_skullstrip_T1.inputs.tissues=[tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
segm_skullstrip_T1.inputs.use_v8struct=False # do not generate SPM8 and higher compatible jobs
segm_skullstrip_T1.inputs.affine_regularization='none'
#segm_skullstrip_T1.inputs.write_deformation_fields=[True, True] # write forward and inverse deformation fields

In [15]:
# use NewSegment from spm to segment unimportant anatomical landmarks and perform "brain-extraction" (TEST FED006)
#for i in images.keys():
for i in images2.keys():
    segm_skullstrip_T1.inputs.channel_files=images2[i]
    for j in subdirs:
        if j.rsplit('/', 1)[1]==i:
            segm_skullstrip_T1.base_dir=j

results=segm_skullstrip_T1.run()
print(results.outputs)

191219-11:27:32,184 nipype.workflow INFO:
	 [Node] Setting-up "segm_skullstrip_T1" in "/VBM/FED006/segm_skullstrip_T1".
191219-11:27:32,190 nipype.workflow INFO:
	 [Node] Running "segm_skullstrip_T1" ("nipype.interfaces.spm.preprocess.NewSegment")
191219-11:31:30,498 nipype.workflow INFO:
	 [Node] Finished "segm_skullstrip_T1".

bias_corrected_images = <undefined>
bias_field_images = <undefined>
dartel_input_images = [[], [], [], ['/VBM/FED006/segm_skullstrip_T1/rc4T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii'], ['/VBM/FED006/segm_skullstrip_T1/rc5T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii'], []]
forward_deformation_field = <undefined>
inverse_deformation_field = <undefined>
modulated_class_images = [[], [], [], [], [], []]
native_class_images = [[], [], [], ['/VBM/FED006/segm_skullstrip_T1/c4T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii'], ['/VBM/FED006/segm_skullstrip_T1/c5T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage

In [230]:
# transform posthoc probability maps for bone and soft tissues into nibabel objects
import nibabel as nib
import re

posttpms=dict()
segmdirs=dict()

# define start of dirname for segmentation
targetnode="segm"

for i in images2.keys():
    for j in subdirs:
        # match keys with subdir list
        if j.rsplit('/',1)[1] == i: 
            for k in os.listdir(j):
                if os.path.isdir(os.path.join(j, k)): # go through content of subdirs and look for "true" directories
                    # regex functions match=exact match of pattern, search=finds elements that contain pattern
                    # regex matches entries in j with targetnode k
                    match=re.match(r'(' + targetnode + ').*', k)
                    # ? This is needed to recognize the attribute group from the regex output
                    if match: 
                        segmdirs[j]=os.path.join(j, match.group(0))


# enter tissue class name from segment.output=native-space drawn from TPM.nii classifications
targettp=["c4T1", "c5T1"]
for i in segmdirs.keys():
    objects=list()
    for j in os.listdir(segmdirs[i]):
        for t in targettp:
            # match entries in j with targettp in t
            match=re.match(r'(' + t + ').*', j)
            # appends to the objects list, if match is found
            if match:
                objects.append(nib.load(os.path.join(segmdirs[i], match.group(0))))
    # connects the objects list to subjects entry in posttpms dict
    posttpms[i]=objects
        
# explore posttpms
print(posttpms["/VBM/FED006"][0:2])

[<nibabel.nifti1.Nifti1Image object at 0x7f55e93707f0>, <nibabel.nifti1.Nifti1Image object at 0x7f55e9370ef0>]


In [232]:
print(posttpms["/VBM/FED006"][1])

<class 'nibabel.nifti1.Nifti1Image'>
data shape (192, 256, 256)
affine: 
[[ 8.97385120e-01 -2.61366367e-03  7.12976456e-02 -9.42629852e+01]
 [-1.32349730e-02  9.12595689e-01  2.14208603e-01 -1.39385727e+02]
 [-6.72004223e-02 -2.14635849e-01  9.09910738e-01 -7.46116333e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 192 256 256   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [1.        0.8999951 0.9375    0.9375    0.        0.        0.
 0.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max   

In [88]:
# create FULL segment node
segment_T1=Node(spm.preprocess.NewSegment(), name='segment_T1')
tissue1 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 1), 2, (True,True), (False, False)) # GM
tissue2 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 2), 2, (True,True), (False, False)) # WM
tissue3 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 3), 2, (True,True), (False, False)) # CSF
tissue4 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 4), 2, (True,False), (False, False)) # bone
tissue5 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 5), 2, (True,False), (False, False)) # soft tissue
tissue6 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 6), 2, (False,False), (False, False)) # air and background
segment_T1.inputs.tissues=[tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
segment_T1.inputs.use_v8struct=False # do not generate SPM8 and higher compatible jobs
segment_T1.inputs.write_deformation_fields=[True, True] # write forward and inverse deformation fields

In [84]:
# use segment_T1 from spm to segment anatomical images (TEST)
#for i in images.keys():
for i in images2.keys():
    print(images2[i])
    segment_T1.inputs.channel_files=images2[i]
    for j in subdirs:
        if j.rsplit('/', 1)[1]==i:
            print(j)
            segment_T1.base_dir=j

results=segment_T1.run()
print(results.outputs)

/VBM/FED006/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii
/VBM/FED006
191218-17:30:49,25 nipype.workflow INFO:
	 [Node] Setting-up "segment_T1" in "/VBM/FED006/segment_T1".
191218-17:30:49,32 nipype.workflow INFO:
	 [Node] Running "segment_T1" ("nipype.interfaces.spm.preprocess.NewSegment")
191218-17:34:38,384 nipype.workflow INFO:
	 [Node] Finished "segment_T1".

bias_corrected_images = <undefined>
bias_field_images = <undefined>
dartel_input_images = [['/VBM/FED006/segment_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii'], ['/VBM/FED006/segment_T1/rc2T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii'], ['/VBM/FED006/segment_T1/rc3T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii'], [], [], []]
forward_deformation_field = /VBM/FED006/segment_T1/y_T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii
inverse_deformation_field = /VBM/FED006/segment_T1/iy_T1_MPRAGE_SAG_ISO_0_9_0005

Create the study specific, non-linear GM template